In [6]:
from utils.utils_IO import load_object
from utils.utils_plotting import plot_cams_and_points
from copy import deepcopy


load_dict = load_object("sawtell_points.pickle")
config = load_dict["config"]
cam_group = load_dict["cam_group"]
points_3d = load_dict["points_3d"]
bp_names = config["bp_names"]

skeleton_bp = {}
frame_i = 256

slice_3d = points_3d[frame_i]
for i, bp in enumerate(bp_names):
    skeleton_bp[bp] = points_3d[frame_i, i, :]
skeleton_lines = config.skeleton

cam_names = ["Top", "Front", "Side"]

skel_fig = plot_cams_and_points(
    cam_group=cam_group,
    points_3d=slice_3d,
    point_size=3,
    skeleton_bp=skeleton_bp,
    skeleton_lines=skeleton_lines,
    font_size=20,
    point_colors=config.color_list,
    cam_names=cam_names,
)

xe = 0
ye = -1 * 0.7
ze = -2 * 0.7
scene_camera = dict(
    up=dict(x=0, y=-0.5, z=0),
    eye=dict(x=xe, y=ye, z=ze),
)

skel_fig_layout = deepcopy(skel_fig)
skel_fig_layout.update_layout(scene_camera=scene_camera)
skel_fig["layout"] = skel_fig_layout["layout"]

skel_fig.write_image("fish_recon.eps")